<h2>Project Scenerio</h2>
You are a stock analyst working in the research department of a small<br>
hedge fund.  Your job is to analyze the returns of a pool of securities<br>
and back test the quants strategy of buys and sells.  You need<br>
to load all of the securities the (4) securities in the given folder <br>
and test their strategy.<br>
files:<br>

INTC - Intel Corporation<br>
AMD - Advanced Micro Devices<br>
AAPL - Apple<br>
FB - Meta Platforms, Inc.<br>
corporations = {'INTC': 'Intel Corporation',<br>
'AMD':'Advanced Micro Devices',<br>
'AAPL':'Apple',<br>
'FB':'Meta Platforms, Inc.'}<br>
<br>
NOTE: All trades use the closing price!!!!

<h2>Step 1</h2>
<h4>5 points</h4>
Create a directory and place all of the csv files into it<br>
Create the variable proj_dir = '  <your directory> '<br>
db_name = 'stock_analysis'

In [2]:
# import statments
import glob
import os
import pandas as pd
from sqlalchemy import create_engine
import sqlite3


In [3]:
''' Database, File Directories, and Variables '''
db_name = '/Users/rashaanrichardson/Desktop/SQL/SQL_Projects/stock_analysis'
proj_dir = '/Users/rashaanrichardson/Desktop/SQL/SQL_Projects/'
csv_files = '/Users/rashaanrichardson/Desktop/SQL/SQL_Projects/CSV_Files/'
engine = create_engine('sqlite:///'+ db_name, echo=False)

In [4]:
''' Set up my SQL connections'''
conn = sqlite3.connect(db_name)
c = conn.cursor()


<h2>Step 2</h2>
Create the following tables:<br>
1 - security_master<br>
symbol<br>
trade_date<br>
open_price<br>
high_price<br>
low_price<br>
adj_close<br>
close_price<br>
volume<br>
<br>
2 - security_info<br>
symbol<br>
name  Max of 25 characters<br>
research_notes Max of 255 characters<br>
<br>
3 - security_log<br>
transaction_id  Character max 20 alpha numeric<br>
trade_date<br>
symbol<br>
trans_type   - (will contain only 'BUY' or 'SELL')<br>
shares   (Will only contain whole numbers)<br>
cost - Will hold dollar amounts<br>
balance - Will hold dollar amounts<br>

In [9]:
''' Deletes all of the info in our tables. This is useful if we rerun this operation'''

# c.execute('''DELETE IF EXISTS FROM Security_Master;''')
# c.execute('''DELETE IF EXISTS FROM Security_Info;''')

''' Creates all of our tables '''

# Sec Master table
c.execute('''CREATE TABLE IF NOT EXISTS Security_Master (
    symbol VARCHAR(10), trade_date DATE, open_price REAL, high_price REAL, low_price REAL, adj_close REAL, close_price REAL, volume REAL)
''')

# Sec Info table 
c.execute('''CREATE TABLE IF NOT EXISTS Security_INFO(
symbol VARCHAR(10), name VARCHAR(25), research_notes TEXT(255)) 
''')

# Security Log table
c.execute('''CREATE TABLE IF NOT EXISTS Security_Log(
transaction_id VARCHAR(20), trade_date DATE, symbol VARCHAR(10), trans_type CHAR(4), shares INT, cost FLOAT(16),balance FLOAT(16))
''')

conn.commit() #Saves the tables to the db
# conn.close()

<h2>Step 3</h2>
Load the security master table<br>
Your code should read whatever files that are csv files in the directory<br>
Use the dictionary below to populate the security_info table:<br>
corporations = {'INTC': 'Intel Corporation',
'AMD':'Advanced Micro Devices',
'AAPL':'Apple',
'FB':'Meta Platforms, Inc.'}

In [12]:
'''Setting up our DataFrames. The data needs to be cleaned. Will be inserted into the Tables'''

csv_files = '/Users/rashaanrichardson/Desktop/SQL/SQL_Projects/CSV_Files'
# c.execute('''DELETE FROM Security_Master''')

# List all the files in the specified directory using glob.glob()
for file in glob.glob(os.path.join(csv_files, '*.csv')):
    try:
        # get the symbol 
        stock_name = os.path.basename(file).removesuffix('.csv')

        # create the dataframe 
        df = pd.read_csv(file)
        df.rename(columns= {'Adj Close':'Adj_Close'},inplace=True)
        df['Symbol'] = stock_name

        # reorder the column names
        df = df[['Symbol','Date','Open','High','Low','Close','Adj_Close','Volume']]

        # round off the values of the columns
        df[['Open','High','Low','Close','Adj_Close']] = df[['Open','High','Low','Close','Adj_Close']].apply(lambda x:x.round(2))

        # upload the dataframes into the sql table
        df.to_sql('Security_Master', con=engine, if_exists='append', index=False)
        # print(pd.read_sql('SELECT DISTINCT Symbol FROM Security_Master', con=engine))
    except Exception as e:
        print(f"Error processing file {file}: {e}")
print(pd.read_sql('SELECT DISTINCT Symbol FROM Security_Master', con=engine))



  Symbol
0   AAPL
1   INTC
2    AMD
3     FB


<h2>Step 4</h2>

Show data has been added 

In [28]:
# df[['Symbol','Date','Open','High','Low','Close','Adj_Close','Volume']]
c.execute('''SELECT DISTINCT Symbol FROM Security_Master ''')
rows = c.fetchall()
for row in rows:
    print(row)

('AAPL',)
('INTC',)
('AMD',)
('FB',)


<h2>Step 5</h2>

Print the average closing share price for each security in the year 2019<br>
Print the average open for each security in April 2020<br>

In [35]:
# df[['Symbol','Date','Open','High','Low','Close','Adj_Close','Volume']]
# c.execute('''SELECT DISTINCT Symbol FROM Security_Master ''')

c.execute('''SELECT Symbol, Round(AVG(Adj_Close),2) AS newData
             FROM Security_Master
             WHERE Date LIKE '2019%'
             GROUP BY Symbol
            ''')

rows = c.fetchall()
for row in rows:
    print(row)

c.execute('''SELECT Symbol, Round(AVG(Open),2) AS avgOpen
             FROM Security_Master
             WHERE Date LIKE '2020-04%'
             GROUP BY Symbol
            ''')

rows = c.fetchall()
for row in rows:
    print(row)

('AAPL', 51.0)
('AMD', 29.94)
('FB', 181.64)
('INTC', 48.32)
('AAPL', 67.95)
('AMD', 52.03)
('FB', 177.12)
('INTC', 58.22)


<h2>Step 7</h2>
<h4>5 points</h4>
Print the max and min closing share price for each security in the year 2020<br>

In [37]:
c.execute('''SELECT Symbol, MAX(Adj_CLose),MIN(Adj_Close) AS share_price
            FROM Security_Master
            GROUP BY Symbol
            ''')
rows = c.fetchall()
for row in rows:
    print(row)

('AAPL', 156.46, 34.51)
('AMD', 150.16, 16.65)
('FB', 382.18, 124.06)
('INTC', 66.94, 40.45)


<h2>Step 8</h2>
<h4>15 points</h4>
Alter the trading strategy below to update the security_log table<br>
transaction_id = YYYYMMDD_symbol_random number, 100 up to 999<br>
trade_date <br>
symbol <br>
trans_type BUY OR SELL<br>
shares number of shares<br>
cost =  -1 * close_price * shares if a BUY, close_price * shares if a SELL<br>
<br>
<code>
sql = '''
SELECT symbol,trade_date,close_price - open_price as diff,close_price 
FROM security_master 
WHERE symbol = 'AMD'; '''
curr.execute(sql)
rows = curr.fetchall()
up_days = 0
down_days = 0
tot_val = 0
tot_shares = 0
share_size = 10000
for row in rows:
    symbol = row[0]
    y,m,d = str(row[1].year), ('00'+str(row[1].month))[-2:], ('00'+str(row[1].day))[-2:]
    transaction_id = f'{y}{m}{d}_{row[0]}_{str(random.randint(100,1000))}'
    trade_date = row[1]    
    '''
    if closed up for 5 consecutive days SELL 10000 shares
    sell if down for 5 consecute days '''
    if row[2] > 0:
        up_days += 1
        down_days = 0
    else:
        up_days = 0
        down_days += 1
    if up_days >= 5:
        down_days = 0
        '''SELL signal'''
        if tot_shares > 0:
            factor = 2 if tot_shares - (share_size*2) > 0 else 1
            tot_val += ((share_size*factor) * row[3])
            tot_shares -= (share_size*factor)
            # = YYYYMMDD_symbol_random number, 100 up to 999<br>
            print(f'****** SELL HERE {row[0]} {row[1]} {row[3]}  - {tot_shares} {tot_val}') 
            '''----- ADD CODE BELOW --------------'''
            
    if down_days >= 5:
        up_days = 0
        down_days = 0
        '''buy signal'''
        tot_val += (-1 * share_size * row[3])
        tot_shares += share_size
        print(f'****** BUY HERE {row[0]} {row[1]} {row[3]} - {tot_shares} {tot_val}')        
        '''----- ADD CODE BELOW --------------'''
</code>

<h2>Step 9</h2>
<h4>10 points</h4>
If you missed the update on step 8 above manual run the following inserts<br>
Then return the symbol,stock price,trans_type For all 2020 trades<br>

INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190207_AMD_205','2019-02-07','AMD','BUY',
        10000,-226700.0,-226700.0)

INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190328_AMD_354','2019-03-28','AMD','BUY',
        20000,-250599.99,-477299.99)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190625_AMD_336','2019-06-25','AMD','BUY',
        30000,-288600.01,-765900.0)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20190709_AMD_348','2019-07-09','AMD','SELL',
            10000,331500.02,-102899.96)
            
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20190710_AMD_191','2019-07-10','AMD','SELL',
            0,337900.01,235000.05)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20190806_AMD_645','2019-08-06','AMD','BUY',
        10000,-288600.01,-53599.96)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20191114_AMD_174','2019-11-14','AMD','SELL',
            0,383499.98,329900.02)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20200501_AMD_727','2020-05-01','AMD','BUY',
        10000,-498800.01,-168899.99)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20200930_AMD_691','2020-09-30','AMD','SELL',
            0,819899.98,650999.99)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20201020_AMD_135','2020-10-20','AMD','BUY',
        10000,-815599.98,-164599.99)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20201222_AMD_392','2020-12-22','AMD','BUY',
        20000,-931600.04,-1096200.03)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210119_AMD_612','2021-01-19','AMD','BUY',
        30000,-894499.97,-1990700.0)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210128_AMD_454','2021-01-28','AMD','BUY',
        40000,-875199.97,-2865899.97)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210222_AMD_258','2021-02-22','AMD','BUY',
        50000,-853700.03,-3719600.0)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210308_AMD_803','2021-03-08','AMD','BUY',
        60000,-739599.99,-4459199.99)
        
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210503_AMD_543','2021-05-03','AMD','BUY',
        70000,-785500.03,-5244700.02)

INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210528_AMD_974','2021-05-28','AMD','SELL',
            50000,800800.02,-3643099.98)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210716_AMD_346','2021-07-16','AMD','BUY',
        60000,-858899.99,-4501999.97)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210723_AMD_399','2021-07-23','AMD','SELL',
            40000,921500.02,-2658999.93)
            
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210803_AMD_858','2021-08-03','AMD','SELL',
            20000,1125599.98,-407799.97)
            
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210804_AMD_569','2021-08-04','AMD','SELL',
            10000,1187699.97,779900.0)
            
INSERT INTO security_log
        (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
        VALUES 
        ('20210913_AMD_796','2021-09-13','AMD','BUY',
        20000,-1048000.03,-268100.03)
        
INSERT INTO security_log
            (transaction_id,trade_date,symbol,trans_type,shares,cost,balance)
            VALUES 
            ('20210927_AMD_123','2021-09-27','AMD','SELL',
            10000,1081600.04,813500.01)

In [46]:
c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20190207_AMD_205', '2019-02-07', 'AMD', 'BUY', 10000, -226700.0, -226700.0)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20190328_AMD_354', '2019-03-28', 'AMD', 'BUY', 20000, -250599.99, -477299.99)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20190625_AMD_336', '2019-06-25', 'AMD', 'BUY', 30000, -288600.01, -765900.0)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20190709_AMD_348', '2019-07-09', 'AMD', 'SELL', 10000, 331500.02, -102899.96)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20190710_AMD_191', '2019-07-10', 'AMD', 'SELL', 0, 337900.01, 235000.05)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20190806_AMD_645', '2019-08-06', 'AMD', 'BUY', 10000, -288600.01, -53599.96)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20191114_AMD_174', '2019-11-14', 'AMD', 'SELL', 0, 383499.98, 329900.02)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20200501_AMD_727', '2020-05-01', 'AMD', 'BUY', 10000, -498800.01, -168899.99)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20200930_AMD_691', '2020-09-30', 'AMD', 'SELL', 0, 819899.98, 650999.99)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20201020_AMD_135', '2020-10-20', 'AMD', 'BUY', 10000, -815599.98, -164599.99)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20201222_AMD_392', '2020-12-22', 'AMD', 'BUY', 20000, -931600.04, -1096200.03)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210119_AMD_612', '2021-01-19', 'AMD', 'BUY', 30000, -894499.97, -1990700.0)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210128_AMD_454', '2021-01-28', 'AMD', 'BUY', 40000, -875199.97, -2865899.97)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210222_AMD_258', '2021-02-22', 'AMD', 'BUY', 50000, -853700.03, -3719600.0)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210308_AMD_803', '2021-03-08', 'AMD', 'BUY', 60000, -739599.99, -4459199.99)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210503_AMD_543', '2021-05-03', 'AMD', 'BUY', 70000, -785500.03, -5244700.02)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210528_AMD_974','2021-05-28','AMD','SELL',
            50000,800800.02,-3643099.98)''')
c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210528_AMD_974', '2021-05-28', 'AMD', 'SELL', 50000, 800800.02, -3643099.98)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210716_AMD_346', '2021-07-16', 'AMD', 'BUY', 60000, -858899.99, -4501999.97)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210723_AMD_399', '2021-07-23', 'AMD', 'SELL', 40000, 921500.02, -2658999.93)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210803_AMD_858', '2021-08-03', 'AMD', 'SELL', 20000, 1125599.98, -407799.97)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210804_AMD_569', '2021-08-04', 'AMD', 'SELL', 10000, 1187699.97, 779900.0)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210913_AMD_796', '2021-09-13', 'AMD', 'BUY', 20000, -1048000.03, -268100.03)''')

c.execute('''INSERT INTO security_log
            (transaction_id, trade_date, symbol, trans_type, shares, cost, balance)
            VALUES 
            ('20210927_AMD_123', '2021-09-27', 'AMD', 'SELL', 10000, 1081600.04, 813500.01)''')


In [47]:
c.execute('''SELECT * FROM Security_Log''')
rows = c.fetchall()
for row in rows:
    print(row)

('20190207_AMD_205', '2019-02-07', 'AMD', 'BUY', 10000, -226700.0, -226700.0)
('20190328_AMD_354', '2019-03-28', 'AMD', 'BUY', 20000, -250599.99, -477299.99)
('20190625_AMD_336', '2019-06-25', 'AMD', 'BUY', 30000, -288600.01, -765900.0)
('20190709_AMD_348', '2019-07-09', 'AMD', 'SELL', 10000, 331500.02, -102899.96)
('20190710_AMD_191', '2019-07-10', 'AMD', 'SELL', 0, 337900.01, 235000.05)
('20190806_AMD_645', '2019-08-06', 'AMD', 'BUY', 10000, -288600.01, -53599.96)
('20191114_AMD_174', '2019-11-14', 'AMD', 'SELL', 0, 383499.98, 329900.02)
('20200501_AMD_727', '2020-05-01', 'AMD', 'BUY', 10000, -498800.01, -168899.99)
('20200930_AMD_691', '2020-09-30', 'AMD', 'SELL', 0, 819899.98, 650999.99)
('20201020_AMD_135', '2020-10-20', 'AMD', 'BUY', 10000, -815599.98, -164599.99)
('20201222_AMD_392', '2020-12-22', 'AMD', 'BUY', 20000, -931600.04, -1096200.03)
('20210119_AMD_612', '2021-01-19', 'AMD', 'BUY', 30000, -894499.97, -1990700.0)
('20210128_AMD_454', '2021-01-28', 'AMD', 'BUY', 40000, -8

<h2>Step 10</h2>
<h4>10 points</h4>
If you bought 100,000 shares of FB the first trading day of 2020 and sold<br>
100,000 shares on the last day of 2020. How much would you have gained or lost?

<h2>Bonus Question</h2>
<h4>25 points</h4>
Write a program that buys 10000 shares of INTC every time the volume<br>
is greater than 10% for the average daily volume for 10 days and sell 10000 shares <br>
when the average volume for the past 5 days.  The initial buy should be when the shares <br>
have had 5 consecutive positive trading days